In [1]:
import os
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_community.vectorstores.chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings

In [2]:
CHROMA_PATH = os.getenv('CHROMA_PATH', 'chroma_embeddings')
COLLECTION_NAME = os.getenv('COLLECTION_NAME', 'local-rag')
TEXT_EMBEDDING_MODEL = os.getenv('TEXT_EMBEDDING_MODEL', 'nomic-embed-text')  # Default model
TEMP_FOLDER = os.getenv('TEMP_FOLDER', './_temp')
embedding = OllamaEmbeddings(model=TEXT_EMBEDDING_MODEL, show_progress=True)


C:\Users\somaa\AppData\Local\Temp\ipykernel_43200\2241661458.py:5: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding = OllamaEmbeddings(model=TEXT_EMBEDDING_MODEL, show_progress=True)


In [3]:
db = Chroma(
        collection_name=COLLECTION_NAME,
        persist_directory=CHROMA_PATH,
        embedding_function=embedding
    )

C:\Users\somaa\AppData\Local\Temp\ipykernel_43200\454650042.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(


In [4]:
LLM_MODEL = "llama3.2:3b-instruct-q5_K_S"


In [5]:
def get_prompt():
    QUERY_PROMPT = PromptTemplate(
        input_variables=["question"],
        template="""You are an AI assistant. Generate five variations of the user's question to enhance document retrieval from a vector database. Also Provide alternatives separated by newlines.
        Original question: {question}""",
    )

    template = """ Answer the question based ONLY on the provided context:
    {context}
    Question: {question}
    """
    prompt = ChatPromptTemplate.from_template(template)
    return QUERY_PROMPT, prompt

In [9]:
def query(input):
    if input:
        # Initialize the language model
        llm = ChatOllama(model="llama3.2:3b-instruct-q5_K_S")
        QUERY_PROMPT = PromptTemplate(
        input_variables=["question"],
        template="""You are an AI assistant. Generate five variations of the user's question to enhance document retrieval from a vector database. Also Provide alternatives separated by newlines.
        Original question: {question}""",
        )
        # Get the vector database instance
        template = """ Answer the question based ONLY on the provided context:
        {context}
        Question: {question}
        """
        prompt = ChatPromptTemplate.from_template(template)
        # Get the query and answer prompts
        QUERY_PROMPT, prompt = get_prompt()
        CHROMA_PATH = os.getenv('CHROMA_PATH', 'chroma_embeddings')
        COLLECTION_NAME = os.getenv('COLLECTION_NAME', 'local-rag')
        TEXT_EMBEDDING_MODEL = os.getenv('TEXT_EMBEDDING_MODEL', 'nomic-embed-text')  # Default model
        TEMP_FOLDER = os.getenv('TEMP_FOLDER', './_temp')
        embedding = OllamaEmbeddings(model=TEXT_EMBEDDING_MODEL, show_progress=True)
        db = Chroma(
        collection_name=COLLECTION_NAME,
        persist_directory=CHROMA_PATH,
        embedding_function=embedding
        )
        # Multi-document retriever to retrieve from multiple relevant documents
        retriever = MultiQueryRetriever.from_llm(
            retriever=db.as_retriever(search_type="similarity", search_kwargs={"k": 3}),
            llm=llm,
            prompt=QUERY_PROMPT
        )

        # Processing chain
        chain = (
            {"context": retriever, "question": RunnablePassthrough()}
            | prompt
            | llm
            | StrOutputParser()
        )

        response = chain.invoke(input)
        return response

    return None

In [10]:
queries = [
    "What are the differences in the business of Tesla and Uber?",
]

In [11]:
for q in queries:
    print(f"Query: {q}")
    answer = query(q)
    print(f"Answer: {answer}")
    print("-" * 50)

Query: What are the differences in the business of Tesla and Uber?















llamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.10s/it]

Answer: A question about two well-known companies!

Tesla and Uber are both tech giants in their respective industries, but they have distinct business models and focuses. Here are some key differences:

**1. Products/Services:**
	* Tesla: Electric vehicles (EVs), clean energy solutions, solar power systems, and energy storage products. They aim to accelerate the world's transition to sustainable energy.
	* Uber: Ride-hailing and transportation network services (TNS). They connect passengers with drivers for on-demand rides, food delivery, and other transportation needs.

**2. Business Model:**
	* Tesla: Focuses on manufacturing and selling EVs, with a strategy of disrupting the automotive industry by offering high-performance, sustainable vehicles.
	* Uber: Operates as a platform that connects riders with drivers through an app, generating revenue primarily from commission-based fares.

**3. Revenue Streams:**
	* Tesla: Primarily generates revenue from vehicle sales, energy storage pr